In [2]:
### import libraries
import pandas as pd
import numpy as np
import tensorflow as tf

## import train/test data
df = pd.read_excel('data/data.xlsx', sheet_name='Labelled_Data')
df = df[['Tweet Text','Is_Bot ( 1 for Bot / 0 for Human)']]
df = df.rename(columns={'Tweet Text': "tweet", 'Is_Bot ( 1 for Bot / 0 for Human)': "target"})

#df2 = pd.read_excel('data/data.xlsx', sheet_name='Test_Data')
#df2 = df2[['Tweet Text','Is_Bot ( 1 for Bot / 0 for Human)']]
#df2 = df2.rename(columns={'Tweet Text': "tweet", 'Is_Bot ( 1 for Bot / 0 for Human)': "target"})



In [19]:
data = pd.read_csv('data/20_Percent_Final-Eval-NOLabels.csv', delimiter = ',')
data = data[['Screen Name','Tweet Text']]
data = data.rename(columns={'Tweet Text': "tweet"})


In [4]:
sentences_train = df.tweet
sentences_test = data.tweet
y_train = df.target
#y_test = data.

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(sentences_train)
vect = tokenizer.texts_to_sequences(data.tweet)
vect = pad_sequences(vect, padding='post', maxlen=100)
X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)
vocab_size = len(tokenizer.word_index) + 1



Using TensorFlow backend.


In [5]:
#max_len = max([len(i) for i in X_train])
#print(max_len)

maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


from keras.models import Sequential
from keras import layers
from keras.layers import Dense, LSTM

embedding_dim = 1000

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size,output_dim=embedding_dim,input_length=maxlen))
model.add(LSTM(64, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

W1002 16:24:07.816340 11048 deprecation_wrapper.py:119] From C:\Users\ELYES\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1002 16:24:07.848050 11048 deprecation_wrapper.py:119] From C:\Users\ELYES\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1002 16:24:07.853025 11048 deprecation_wrapper.py:119] From C:\Users\ELYES\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1002 16:24:08.077669 11048 deprecation_wrapper.py:119] From C:\Users\ELYES\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1002 16:24:08.091703 11048 deprecation.py:506

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 1000)         38391000  
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                272640    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 38,663,705
Trainable params: 38,663,705
Non-trainable params: 0
_________________________________________________________________


In [6]:
from sklearn.utils import class_weight
weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)


In [15]:
## Fit the model
model.fit(X_train, y_train,epochs=8, validation_split=0.3, batch_size=128)

#model.fit(X_train, y_train, validation_split=0.3, epochs=5,class_weight=weights)

Train on 12868 samples, validate on 5516 samples
Epoch 1/8
12868/12868 [==============================] - 33s 3ms/step - loss: 0.6744 - acc: 0.5976 - val_loss: 0.5317 - val_acc: 1.0000
Epoch 2/8
12868/12868 [==============================] - 36s 3ms/step - loss: 0.6743 - acc: 0.5976 - val_loss: 0.4807 - val_acc: 1.0000
Epoch 3/8
12868/12868 [==============================] - 38s 3ms/step - loss: 0.6742 - acc: 0.5976 - val_loss: 0.5156 - val_acc: 1.0000
Epoch 4/8
12868/12868 [==============================] - 38s 3ms/step - loss: 0.6739 - acc: 0.5985 - val_loss: 0.5133 - val_acc: 1.0000
Epoch 5/8
12868/12868 [==============================] - 38s 3ms/step - loss: 0.3253 - acc: 0.8879 - val_loss: 0.1080 - val_acc: 0.9839
Epoch 6/8
12868/12868 [==============================] - 36s 3ms/step - loss: 0.2515 - acc: 0.9292 - val_loss: 0.1349 - val_acc: 0.9868
Epoch 7/8
12868/12868 [==============================] - 36s 3ms/step - loss: 0.2837 - acc: 0.9133 - val_loss: 0.2026 - val_acc: 0.9873

In [16]:
data = pd.read_csv('data/20_Percent_Final-Eval-NOLabels.csv', delimiter = ',')
data = data[['Tweet ID','Tweet Text']]
data = data.rename(columns={'Tweet Text': "tweet"})



In [17]:
## Apply model on sub. Data

vect = tokenizer.texts_to_sequences(data.tweet)
vect = pad_sequences(vect, padding='post', maxlen=100)

prediction = model.predict_classes(vect)



In [20]:
submission = pd.concat([data['Screen Name'], pd.DataFrame(prediction,columns=['Is_Bot ( 1 for Bot / 0 for Human)'])],axis=1)

## Save submission to csv
submission.to_csv("team_ANLP_submission.csv", index=False)